# COVID-19 - A Study of Imperfect Data

### Step 1: Gather the virus data (confirmed, death, recovered) from the JH github repo and combine

In [6]:
# COVID-19 github files

fbase = r'C:/Users/jshaf/GitHub/COVID-19/csse_covid_19_data/csse_covid_19_time_series/'

# Filenames changed
#f1 = fbase + r'time_series_19-covid-Confirmed.csv'
#f2 = fbase + r'time_series_19-covid-Deaths.csv'
#f3 = fbase + r'time_series_19-covid-Recovered.csv'

f1 = fbase + r'time_series_covid19_confirmed_global.csv'
f2 = fbase + r'time_series_covid19_deaths_global.csv'
f3 = fbase + r'time_series_covid19_recovered_global.csv'

In [7]:
# Versions of this notebook
# v1 - Assemble the time series for confirmed, deaths, and recoveries into a single, 
#      long-file format.
# v2 - Begin checking that the conditions for merge don't drop data; place names are
#      changing, e.g. from just US state to county, state codes.

In [8]:
import pandas as pd

In [9]:
df_confirmed = pd.read_csv(f1)
df_deaths    = pd.read_csv(f2)
df_recovered = pd.read_csv(f3)

In [10]:
type(df_confirmed)

pandas.core.frame.DataFrame

In [11]:
# Check that the location columns will line up for merge later
df_locations = pd.DataFrame(columns=['provincestate_confirmed','countryregion_confirmed'])
#df_confirmed.columns

df_locations[['provincestate_confirmed','countryregion_confirmed']] = df_confirmed[['Province/State','Country/Region']]
df_locations[['provincestate_deaths',   'countryregion_deaths']]    = df_deaths[['Province/State','Country/Region']]
df_locations[['provincestate_recovered','countryregion_recovered']] = df_recovered[['Province/State','Country/Region']]

In [12]:
df_locations.fillna('no text',inplace=True)
df_locations.head()

,provincestate_confirmed,countryregion_confirmed,provincestate_deaths,countryregion_deaths,provincestate_recovered,countryregion_recovered
0,no text,Afghanistan,no text,Afghanistan,no text,Afghanistan
1,no text,Albania,no text,Albania,no text,Albania
2,no text,Algeria,no text,Algeria,no text,Algeria
3,no text,Andorra,no text,Andorra,no text,Andorra
4,no text,Angola,no text,Angola,no text,Angola


In [13]:
def check_match(s: pd.core.series.Series) -> pd.core.series.Series:
    '''Check that all Province/State matches in c, r, and d files
       Check that all Country/Region matches in c, r, and d files
       Return a flag True if isMisMatch
    '''

    if ((s['provincestate_confirmed'] == s['provincestate_recovered']) & \
        (s['provincestate_confirmed'] == s['provincestate_deaths']) &\
        (s['countryregion_confirmed'] == s['countryregion_recovered']) & \
        (s['countryregion_confirmed'] == s['countryregion_deaths'])):
        s['isMisMatch'] = False 
    else:
        print("mismatch on row!\n{}\n".format(s))
        s['isMisMatch'] = True
    return s


df_locations = df_locations.apply(check_match, axis=1)

# 2020.03.15 This check flagged that 4 items were out of order: Aruba, Canada (Diamond Princess), Guinea, Kenya
#   However, on inspection, all 4 have line items they're just out of order.
#   TODO: Today, let this go, but this check should be refactored so that it checks content and ignores order.

print("\n\nNumber of mismatched location columns: {}".format(df_locations['isMisMatch'].sum()))
print(" This indicates that the 3 files (confirmed, recovered, deaths) may have different countries listed.")
print(" Depending on how the data is merged, some data could be lost.")

mismatch on row!
provincestate_confirmed    no text
countryregion_confirmed      Benin
provincestate_deaths       no text
countryregion_deaths         Benin
provincestate_recovered    no text
countryregion_recovered     Belize
Name: 24, dtype: object

mismatch on row!
provincestate_confirmed    no text
countryregion_confirmed     Bhutan
provincestate_deaths       no text
countryregion_deaths        Bhutan
provincestate_recovered    no text
countryregion_recovered      Benin
Name: 25, dtype: object

mismatch on row!
provincestate_confirmed    no text
countryregion_confirmed    Bolivia
provincestate_deaths       no text
countryregion_deaths       Bolivia
provincestate_recovered    no text
countryregion_recovered     Bhutan
Name: 26, dtype: object

mismatch on row!
provincestate_confirmed                   no text
countryregion_confirmed    Bosnia and Herzegovina
provincestate_deaths                      no text
countryregion_deaths       Bosnia and Herzegovina
provincestate_recovered    


mismatch on row!
provincestate_confirmed        no text
countryregion_confirmed    Netherlands
provincestate_deaths           no text
countryregion_deaths       Netherlands
provincestate_recovered        no text
countryregion_recovered          Niger
Name: 169, dtype: object

mismatch on row!
provincestate_confirmed        no text
countryregion_confirmed    New Zealand
provincestate_deaths           no text
countryregion_deaths       New Zealand
provincestate_recovered        no text
countryregion_recovered        Nigeria
Name: 170, dtype: object

mismatch on row!
provincestate_confirmed            no text
countryregion_confirmed          Nicaragua
provincestate_deaths               no text
countryregion_deaths             Nicaragua
provincestate_recovered            no text
countryregion_recovered    North Macedonia
Name: 171, dtype: object

mismatch on row!
provincestate_confirmed    no text
countryregion_confirmed      Niger
provincestate_deaths       no text
countryregion_deaths  

In [14]:
def wide_to_long(dfin: pd.core.frame.DataFrame, case_type: str) -> pd.core.frame.DataFrame:
    '''Melt a wide time series into a long data frame
        INPUT:      dfin      DataFrame
        OUTPUT:               DataFrame
    '''
    orig_var_list = dfin.columns
    id_varlist = ['Province/State','Country/Region','Lat','Long']  # known from previous work with dataset
    
    val_varlist = orig_var_list.drop(id_varlist)
    
    dfc_melted = pd.melt(dfin,id_vars=id_varlist,value_vars=val_varlist)
    
    # case_type is expected to be one of Confirmed, Deaths, Recovered
    dfc_melted.columns=(['Province/State','Country/Region','Lat','Long','Date',case_type])
    
    return dfc_melted

In [15]:
dwide_c = wide_to_long(df_confirmed,"Confirmed")
dwide_c.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.0000,65.0000,1/22/20,0
1,NaN,Albania,41.1533,20.1683,1/22/20,0
2,NaN,Algeria,28.0339,1.6596,1/22/20,0
3,NaN,Andorra,42.5063,1.5218,1/22/20,0
4,NaN,Angola,-11.2027,17.8739,1/22/20,0


In [16]:
dwide_d = wide_to_long(df_deaths,"Deaths")
dwide_d.head()

,Province/State,Country/Region,Lat,Long,Date,Deaths
0,NaN,Afghanistan,33.0000,65.0000,1/22/20,0
1,NaN,Albania,41.1533,20.1683,1/22/20,0
2,NaN,Algeria,28.0339,1.6596,1/22/20,0
3,NaN,Andorra,42.5063,1.5218,1/22/20,0
4,NaN,Angola,-11.2027,17.8739,1/22/20,0


In [17]:
dwide_r = wide_to_long(df_recovered,"Recovered")
dwide_r.head()

,Province/State,Country/Region,Lat,Long,Date,Recovered
0,NaN,Afghanistan,33.0000,65.0000,1/22/20,0
1,NaN,Albania,41.1533,20.1683,1/22/20,0
2,NaN,Algeria,28.0339,1.6596,1/22/20,0
3,NaN,Andorra,42.5063,1.5218,1/22/20,0
4,NaN,Angola,-11.2027,17.8739,1/22/20,0


In [18]:
# merge the framges

d2 = dwide_c.merge(dwide_d,on=['Province/State','Country/Region','Lat','Long','Date'],\
                   how='left')

d2 = d2.merge(dwide_r,on=['Province/State','Country/Region','Lat','Long','Date'],\
              how='left')

In [19]:
d2.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.0000,65.0000,1/22/20,0,0,0.0
1,NaN,Albania,41.1533,20.1683,1/22/20,0,0,0.0
2,NaN,Algeria,28.0339,1.6596,1/22/20,0,0,0.0
3,NaN,Andorra,42.5063,1.5218,1/22/20,0,0,0.0
4,NaN,Angola,-11.2027,17.8739,1/22/20,0,0,0.0


In [20]:
# convert "Mainland China" to "China"
d2['Country/Region'] = d2['Country/Region'].replace({'Mainland China':'China', 'US':'United States'})

In [21]:
# Save to file for further data analysis, e.g. combining with other information.
import time
ts = time.gmtime()
mytimestamp = time.strftime("%Y-%m-%d_%H%M%S", ts)
#print(time.strftime("%Y-%m-%d %H:%M:%S", ts))

## Attention -- this is UTC time

print(mytimestamp)

# 2020-01-03 09:25:18
fname = "consolidated_COVID-19_" + mytimestamp + "UTC.csv"
print(fname)


2020-03-29_150114
consolidated_COVID-19_2020-03-29_150114UTC.csv


In [22]:
d2.to_csv(fname,index=False)

In [23]:
# also declare it to be "most recent" in preparation for the next step
d2.to_csv('most_recent_COVID-19.csv',index=False)